# **ProductLens** - An AI-powered product comparison tool

## Overview

**ProductLens** helps users compare products and services based on *their* priorities (e.g., privacy, price, ease of setup, brand). It demonstrates a lightweight agentic workflow: user intent → planning → parallel research → comparison → recommendation. 

<br>

![ProductLens](../img/productlens.png)

**Flow**

A central manager orchestrates planning, parallel product research, and final comparison, ensuring each agent does one focused job and produces an explainable, decision-ready result.

## Requirements

**Required**
- Python 3.10+, gradio, python-dotenv, pydantic, asyncio

**LLM / Search (choose one)**
- **($$) OpenAI**
    - OPENAI_API_KEY in .env
    - Uses OpenAI models + `WebSearchTool`
    - Traces logs (no charges for this one)
- **(Free) Ollama**
    - Ollama model in local.
    - Ollama API KEY (need to create an account)
    - Uses [Ollama's web search API](https://docs.ollama.com/capabilities/web-search)

**Configurable**
- MODEL (e.g., `gpt-4o-mini`, local Ollama like `gpt-oss`)


# Setup

- Create .env with chosen API key

### Imports

In [2]:
import asyncio
from agents import Agent, trace, OpenAIChatCompletionsModel, Model, ModelProvider, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from openai import AsyncOpenAI
from pydantic import BaseModel, Field

import os
import requests
from IPython.display import display, Markdown

from dotenv import load_dotenv

In [3]:
load_dotenv(override=True)

True

### Configure Model

Set MODEL variable

#### OpenAI

```python
MODEL = "gpt-5-nano"
```

#### Local Ollama

If using Ollama, you can use OpenAI’s Agents SDK with `gpt-oss` locally ([more info](https://cookbook.openai.com/articles/gpt-oss/run-locally-ollama#pick-your-model)).

We just need a quick setup for compatibility:

In [4]:
class OllamaProvider(ModelProvider):
    def __init__(self):
        self.base_url = "http://localhost:11434/v1"
        self.api_key = "ollama"
    
    def get_async_client(self):
        return AsyncOpenAI(
            base_url=self.base_url, 
            api_key=self.api_key
            )

    def get_model(self, model_name: str) -> Model:
        return OpenAIChatCompletionsModel(
            openai_client=self.get_async_client(), 
            model=model_name
            )        

In [5]:
ollama = OllamaProvider()
MODEL = ollama.get_model(model_name="gpt-oss")

# Complete Flow

**1. Comparison Manager (Orchestrator)**

* Entry point for the user query
* Coordinates all agents
* Manages async execution and data flow
* Owns the end-to-end lifecycle

**2. Planner Agent**

* Parses the user query and priorities
* Identifies relevant products/ecosystems to compare
* Derives evaluation criteria
* Outputs a structured plan (products + criteria)


**3. Product Research Agents (Parallel)**

* Spawned by the Comparison Manager
* Each researches one product
* Uses web search focused on planner criteria
* Returns concise product summaries

**4. Comparator / Decision Agent**

* Invoked by the Comparison Manager
* Normalizes research results
* Evaluates each product against the criteria
* Produces recommendation, table, and tradeoffs


**How They Come Together**

```text
User Query
   ↓
Comparison Manager
   ↓
Planner Agent
   ↓
Parallel Product Research Agents
   ↓
Comparator Agent
   ↓
Final Recommendation
```

**Key Point:**
- Each agent does one job, passing structured outputs forward, resulting in a clear, explainable recommendation.
- Agents reason and specialize; the **Comparison Manager controls execution**.


# Implementation

## Web search tool

- Use either OpenAI's `WebSearchTool` (additional costs)
- Or Ollama's web search tool via HTTP call.

In [6]:
# Ollama's web search
@function_tool
def web_search(product, search_focus) -> list[dict]:
    """Web Search Tool via Ollama API"""

    url = "https://ollama.com/api/web_search"
    headers = {"Authorization": f"Bearer {os.environ.get('OLLAMA_API_KEY_CLOUD')}",
               "Content-Type": "application/json"}
    payload = {"query": f"{product}+{search_focus}"}
    
    response = requests.post(url, 
                             json=payload, 
                             headers=headers)
    
    if response.status_code == 202:
        return response
    else:
        return {"status": "failure", "message": response.text}

## Agents

In [7]:
# Planner Agent

NUM_OF_SEARCHES = 1     # limit searches
INSTRUCTIONS = f"Given a consumer product decision query, identify relevant still in production or operation products/ecosystems,\
derive evaluation criteria from stated priorities, and define {NUM_OF_SEARCHES} search per product."

class ProductPlan(BaseModel):
    products: list[str] = Field(description="A list of products to search.")
    criteria: list[str] = Field(description="A list of priorities for a given product")
    searches: list[str] = Field(description="A list of web searches to perfom for a product based on the evaluation criteria.")

planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model=MODEL,
    output_type=ProductPlan,
)

In [8]:
# Search Agent

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    # tools=[WebSearchTool(search_context_size="low")]          # OpenAI's tool
    tools=[web_search],                                       # Ollama's tool  
    model=MODEL,
    model_settings=ModelSettings(tool_choice="required"),
)

In [9]:
# Comparator Agent

INSTRUCTIONS = (
    "You are a decision assistant. Given structured product research summaries, compare products against user priorities.\n"
    "Normalize differences, rank options, and recommend the best choice.\n"
    "Be concise and decision-oriented."
)


class ComparisonResult(BaseModel):
    summary: str = Field(description="A summary of the findings with a comparison table, including a best for and tradeoffs sections in markdown format.")
    best_for: list[str]
    comparison_table: str = Field(description="A comparison table cleaned up, readable, and structured table in markdown format.")
    tradeoffs: list[str] = Field(description="A list of tradeoffs.")
    sources: list[str] = Field(description="A list of sources used to provide the recommendation.")

comparator_agent = Agent(
    name="Comparator Agent",
    instructions=INSTRUCTIONS,
    model=MODEL,
    output_type=ComparisonResult,
)

## Orchestrator - Comparison Manager

Note: *full code not provided*

In [ ]:

class ComparisonManager:
    """
    Orchestrates the product comparison workflow:
    planning → parallel research → comparison.
    """

    async def run(self, query: str):
        """ Run the product comparison process, yielding the status updates and the recommendation"""
        trace_id = gen_trace_id()

        with trace("Comparison trace", trace_id=trace_id):
            print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}")
            yield f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}"
            
            # 1. Plan
            print("Starting comparison...")
            search_plan = await self.plan_searches(query)
            yield "Researching products identified..."   

            # 2. Parallel research
            research_results = await self.perform_searches(search_plan)
            yield "Research complete. Comparing options..."

            # 3. Compare & decide
            comparison = await self.compare(research_results, search_plan.criteria)
            yield "Product comparison completed."
            
            yield "## Recommendation"
            yield comparison.summary

        # ...

# Results

In [ ]:
manager = ComparisonManager()
query = "Privacy-focused smart doorbells"

results = []
async for update in manager.run(query):
    results.append(update)

results

View trace: https://platform.openai.com/traces/trace?trace_id=trace_8306362de1c84e0ca1ab03b263640cb3
View trace: https://platform.openai.com/traces/trace?trace_id=trace_8306362de1c84e0ca1ab03b263640cb3
Starting comparison...
Planning product searches...
Will perform 4 searches
Researching products identified...
Researching...


[non-fatal] Tracing: server error 503, retrying.


Searching... 1/4 completed
Searching... 2/4 completed
Searching... 3/4 completed
Searching... 4/4 completed
Finished searching
Research complete. Comparing options...
Comparing products...
Finished comparing products.
Product comparison completed.
## Recommendation
Only one product summary is available – Noddy.  The description provides no technical detail, so I cannot confirm that it meets the strict privacy, security, and open‑source requirements you listed.  I recommend collecting more info (spec sheet, firmware repo, hardware teardown) for Noddy and then comparing it to other open‑source doorbell projects (e.g., ESP‑Doorbell, Pi‑Doorbell‑V2, OpenDoorbell).  If Noddy does satisfy TLS, AES‑256 at rest, SD‑card/NAS storage, user‑controlled retention, signed OTA, secure boot, tamper‑proof chassis, minimal telemetry, GDPR compliance, and has a transparent privacy policy, it would be the best fit.  Without that evidence, it currently cannot be endorsed. }


Below results from UI search:

![ProductLens Initial search](../img/productlens-result.png)

![ProductLens result 1](../img/productlens-result1.png)
![ProductLens result 2](../img/productlens-result2.png)